In [1]:
import numpy as np
import pandas as pd

import os
import glob
import librosa
import tqdm
import pickle

DATA_PATH = './data'
join_path = lambda path: os.path.join(DATA_PATH, path)

In [2]:
sampling_rate = 44100 # от оргов контеста
duration = 2 # сколько секунд от записи хотим взять
n_samples = sampling_rate * duration
n_mels=128 # дефолтный параметр в самой либросе

def audio2mel(audio):
    #audio, _ = librosa.effects.trim(audio) # убирает участки тишины из audio, почти не влияет, да и может вредит
    mel = librosa.feature.melspectrogram(
        y=audio,
        sr=sampling_rate,
        n_mels=n_mels, # количество мел-признаков, дефолтное значение для librosa
        n_fft=n_mels * 20, # это ширина окна для FFT
        hop_length=346 * duration, # длина фрейма, n_samples // 128, чтобы получить 128 фреймов
        # ось времени таки есть!
        fmax=sampling_rate // 2, # максимальная частота сигнала, дефолтное значение для librosa
        fmin = 0., # дефолтная наименьшая частота сигнала
    )
    mel = librosa.power_to_db(mel).astype(np.float32) # перевод в дб
    return mel

def get_mels(filepaths, verbose=False):
    data = []
    for fp in tqdm.tqdm(filepaths, disable=not verbose):
        y, _ = librosa.load(fp, sr=sampling_rate)
        y = np.concatenate([[0.] * (n_samples - len(y)), y[:n_samples]])
        data.append(audio2mel(y))
    data = np.array(data)
    return data

In [3]:
# saving train_curated melspectrograms as a single file

train_filepaths = glob.glob(join_path('split/train/*'))
val_filepaths = glob.glob(join_path('split/val/*'))

train_data = get_mels(train_filepaths, verbose=True)
with open(join_path('train_mels.pkl'), 'wb') as f:
    pickle.dump(train_data, f, pickle.HIGHEST_PROTOCOL)

val_data = get_mels(val_filepaths, verbose=True)
with open(join_path('val_mels.pkl'), 'wb') as f:
    pickle.dump(val_data, f, pickle.HIGHEST_PROTOCOL)

100%|██████████| 506/506 [00:05<00:00, 91.90it/s] 


In [17]:
# saving targets

def get_targets(filepaths):
    
    targets = np.array(
        train.loc[[get_filename(fp) for fp in filepaths], 'labels'] \
            .str \
            .split(',') \
            .apply(
                lambda x: [int(label in x) for label in cats]
            ) \
            .tolist()
    )
    
    return targets

get_filename = lambda x: x[-12:]

train = pd.read_csv(join_path('train_curated.csv')).set_index('fname')

cats = list(set(sum(list(train['labels'].str.split(',').values), [])))

train_targets = get_targets(train_filepaths)
val_targets = get_targets(val_filepaths)

np.save(join_path('train_targets.npy'), train_targets)
np.save(join_path('val_targets.npy'), val_targets)